In [3]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist=input_data.read_data_sets('/tmp/data/',one_hot=True)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [4]:
mnist.train.num_examples

55000

In [5]:
mnist.validation.num_examples

5000

In [6]:
mnist.test.num_examples

10000

In [8]:
mnist.train.labels[0]

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.])

In [12]:
batch_size=128
xs,ys=mnist.train.next_batch(batch_size)
print("X shape:",xs.shape)
print("Y shape:",ys.shape)

X shape: (128, 784)
Y shape: (128, 10)


In [24]:
#全连接模型
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

INPUT_NODE=784
OUTPUT_NODE=10
LAYER_NODE=500#隐藏层500
BATCH_SIZE=128#每个batch的大小


#相关参数配置
LEARNING_RATE_BASE=0.8
LEARNING_RATE_DECAY=0.99
REGULARAZTION_RATE=0.0001
TRAINING_STEPS=5000
MOVING_AVERAGE_DECAY=0.99

#定义辅助函数计算前向传播，使用relu作为激活函数
def inference(input_tensor,avg_class,weights1,biases1,weights2,biases2):
    #不使用滑动平均类
    if(avg_class==None):
        layer1=tf.nn.relu(tf.matmul(input_tensor,weights1)+biases1)
        return tf.matmul(layer1,weights2)+biases2
    else:
        #使用滑动平均类
        layer1=tf.nn.relu(tf.matmul(input_tensor,avg_class.average(weights1))+avg_class.average(biases1))
        return tf.matmul(layer1,avg_class.average(weights2)+avg_class.average(biases2))
    
#定义训练过程
def train(mnist):
    x=tf.placeholder(tf.float32,[None,INPUT_NODE],name='X')
    y_=tf.placeholder(tf.float32,[None,OUTPUT_NODE],name='Y')
    
    #生成隐藏层参数
    weight1=tf.Variable(tf.truncated_normal([INPUT_NODE,LAYER_NODE],stddev=1))
    biases1=tf.Variable(tf.constant(0.1,shape=[LAYER_NODE]))
    
    #生成输出层的参数
    weight2=tf.Variable(tf.truncated_normal([LAYER_NODE,OUTPUT_NODE],stddev=0.1))
    biases2=tf.Variable(tf.constant(0.1,shape=[OUTPUT_NODE]))
    
    #计算不含滑动平均类的前向传播结果
    y=inference(x,None,weight1,biases1,weight2,biases2)
    
    #定义训练轮次以及相关的滑动平均类
    global_step=tf.Variable(0,trainable=False)
    variable_averages=tf.train.ExponentialMovingAverage(MOVING_AVERAGE_DECAY,global_step)
    variable_averages_op=variable_averages.apply(tf.trainable_variables())
    average_y=inference(x,variable_averages,weight1,biases1,weight2,biases2)
    #计算交叉熵以及他的平均值
    cross_entropy=tf.nn.sigmoid_cross_entropy_with_logits(logits=y,labels=y_)
    cross_entropy_mean=tf.reduce_mean(cross_entropy)
    
    #损失函数的计算
    regularizer=tf.contrib.layers.l2_regularizer(REGULARAZTION_RATE)
    regulariztion=regularizer(weight1)+regularizer(weight2)
    loss=cross_entropy_mean+regulariztion
    #设置学习率的衰减率
    learning_rate=tf.train.exponential_decay(
        LEARNING_RATE_BASE,
        global_step,
        mnist.train.num_examples/BATCH_SIZE,
        LEARNING_RATE_DECAY,
        staircase=True)
    
    #优化损失函数
    train_step=tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss,global_step=global_step)
    
    # 反向传播更新参数和更新每一个参数的滑动平均值
    with tf.control_dependencies([train_step, variable_averages_op]):
        train_op = tf.no_op(name='train')

    # 计算正确率
    correct_prediction = tf.equal(tf.argmax(average_y, 1), tf.argmax(y_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

    # 初始化回话并开始训练过程。
    with tf.Session() as sess:
        tf.global_variables_initializer().run()
        validate_feed = {x: mnist.validation.images, y_: mnist.validation.labels}
        test_feed = {x: mnist.test.images, y_: mnist.test.labels}

        # 循环的训练神经网络。
        for i in range(TRAINING_STEPS):
            if i % 1000 == 0:
                validate_acc = sess.run(accuracy, feed_dict=validate_feed)
                print("After %d training step(s), validation accuracy using average model is %g " % (i, validate_acc))

            xs, ys = mnist.train.next_batch(BATCH_SIZE)
            sess.run(train_op, feed_dict={x: xs, y_: ys})

        test_acc = sess.run(accuracy, feed_dict=test_feed)
        print(("After %d training step(s), test accuracy using average model is %g" % (TRAINING_STEPS, test_acc)))


# 4. 主程序入口，这里设定模型训练次数为5000次。
def main(argv=None):
    mnist=input_data.read_data_sets('/tmp/data/',one_hot=True)
    train(mnist)
if __name__=='__main__':
    main()


Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
After 0 training step(s), validation accuracy using average model is 0.1102 
After 1000 training step(s), validation accuracy using average model is 0.0976 
After 2000 training step(s), validation accuracy using average model is 0.0958 
After 3000 training step(s), validation accuracy using average model is 0.0958 
After 4000 training step(s), validation accuracy using average model is 0.0958 
After 5000 training step(s), test accuracy using average model is 0.098
